#### Data source
https://opendata.nhsbsa.net/dataset/english-prescribing-data-epd



### Identifying High-Prescription Regions (Clustering)
Goal: Group UK regions based on their prescribing behavior.

#### Load the data

In [2]:
# For data manipulation
import numpy as np
import pandas as pd
import dask.dataframe as dd

# For data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# For displaying all of the columns in dataframes
pd.set_option('display.max_columns', None)

# For data modeling
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from xgboost import plot_importance

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# For metrics and helpful functions
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score,\
f1_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.tree import plot_tree

# For saving models
import pickle

# For deleting unused variables
import gc

#### Data dictionary

| **Column**                          | **Title**                                                        | **Type**   | **Description**                                                                                                                                                                                                                                                                                        |
|-------------------------------------|------------------------------------------------------------------|------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `YEAR_MONTH`                        | Year and Month as YYYYMM                                         | number     | Example: 201401                                                                                                                                                                                                                                                                                          |
| `REGIONAL_OFFICE_NAME`              | Regional Office Name                                             | string     | The name given to a geographical region by NHS England. Each region supports local systems to provide more joined-up care for patients.                                                                                                                                                                   |
| `REGIONAL_OFFICE_CODE`              | Regional Office Code                                             | string     | The unique code used to refer to a Regional Office.                                                                                                                                                                                                                                                     |
| `ICB_NAME`                          | Integrated Care Board (ICB) Name                                 | string     | The name given to a geographical statutory organisation by NHS England that is a smaller division of a Region.                                                                                                                                                                                           |
| `ICB_CODE`                          | Integrated Care Board (ICB) Code                                 | string     | The unique code used to refer to an ICB.                                                                                                                                                                                                                                                                    |
| `PCO_NAME`                          | Primary Care Organisation Name                                   | string     | An NHS organisation that commissions or provides care services involving prescriptions that are dispensed in the community.                                                                                                                                                                            |
| `PCO_CODE`                          | Primary Care Organisation Code                                   | string     | The unique code used to refer to a Primary Care Organisation.                                                                                                                                                                                                                                            |
| `PRACTICE_NAME`                     | Practice Name                                                    | string     | The name of an organisation that employs one or more prescribers who issue prescriptions that may be dispensed in the community.                                                                                                                                                                          |
| `PRACTICE_CODE`                     | Practice Code                                                    | string     | The unique code used to refer to a Practice.                                                                                                                                                                                                                                                               |
| `ADDRESS_1`                         | Address Field 1                                                  | string     | The Address used by a Practice. This data is supplied by Primary Care Support England (PSCE), NHS England ICBs or the SICBL, whenever a new practice is opened or if a change of details is required.                                                                                                   |
| `ADDRESS_2`                         | Address Field 2                                                  | string     | The Address used by a Practice. This data is supplied by Primary Care Support England (PSCE), NHS England ICBs or the SICBL, whenever a new practice is opened or if a change of details is required.                                                                                                   |
| `ADDRESS_3`                         | Address Field 3                                                  | string     | The Address used by a Practice. This data is supplied by Primary Care Support England (PSCE), NHS England ICBs or the SICBL, whenever a new practice is opened or if a change of details is required.                                                                                                   |
| `ADDRESS_4`                         | Address Field 4                                                  | string     | The Address used by a Practice. This data is supplied by Primary Care Support England (PSCE), NHS England ICBs or the SICBL, whenever a new practice is opened or if a change of details is required.                                                                                                   |
| `POSTCODE`                          | Post Code                                                        | string     | The Address used by a Practice. This data is supplied by Primary Care Support England (PSCE), NHS England ICBs or the SICBL, whenever a new practice is opened or if a change of details is required.                                                                                                   |
| `BNF_CHEMICAL_SUBSTANCE`            | British National Formulary (BNF) Chemical Substance Code        | string     | A unique code used to refer to a BNF Chemical Substance. For example, 0501013B0                                                                                                                                                                                                                           |
| `CHEMICAL_SUBSTANCE_BNF_DESCR`      | British National Formulary (BNF) Chemical Substance Description | string     | The name of the main active ingredient in a drug or the type of an appliance. Determined by the British National Formulary (BNF) for drugs, or the NHS BSA for appliances. For example, Amoxicillin                                                                                                     |
| `BNF_CODE`                          | British National Formulary (BNF) Code                            | string     | The unique code used to refer to a BNF Presentation. For example, 0501013B0AAABAB                                                                                                                                                                                                                        |
| `BNF_DESCRIPTION`                   | British National Formulary (BNF) Description                     | string     | The name given to the specific type, strength, and formulation of a drug; or, the specific type of an appliance. For example, Amoxicillin 500mg capsules                                                                                         |
| `BNF_CHAPTER_PLUS_CODE`             | British National Formulary (BNF) Chapter Code                    | string     | The name given to a British National Formulary (BNF) Chapter that includes the prescribed product. Includes the numerical code used to refer to the chapter. For example, 05: Infections                                                                                                             |
| `QUANTITY`                          | Quantity                                                         | number     | The quantity of a medicine, dressing or appliance for which an individual item was prescribed and dispensed, for each BNF Presentation. Example: 28 for Amoxicillin 500mg capsules                                                                                                                   |
| `ITEMS`                             | Items                                                            | number     | The number of times a product appears on a prescription form. Prescription forms include both paper prescriptions and electronic messages.                                                                                                                                                              |
| `TOTAL_QUANTITY`                    | Total Quantity                                                   | number     | The total quantity of a drug or appliance that was prescribed. Calculated by multiplying Quantity by Items. Example: If 2 items of Amoxicillin 500mg capsules with a quantity of 28 were prescribed, total quantity = 56.                            |
| `ADQUSAGE`                          | Average Daily Quantity (ADQ)                                     | number     | The typical daily dose of a medication, prescribed to adult patients by GP Practices. The field shows the quantity prescribed multiplied by the strength, divided by the Average Daily Quantity value.                                                                                                 |
| `NIC`                               | Net Ingredient Cost (NIC)                                        | number     | In GBP. The amount paid using the basic price of the prescribed drug or appliance and the quantity prescribed.                                                                                                                                                                                              |
| `ACTUAL_COST`                       | Actual Cost                                                      | number     | In GBP. The basic cost after adjustment for the national average discount and some payments to the dispenser. The calculation is: Net Ingredient Cost - National Average Discount Percentage + (payment for consumables + out-of-pocket expenses)    |
| `UNIDENTIFIED`                      | Unidentified                                                     | string     | This field shows data from prescription forms that could not be allocated to a Practice.                                                                                                                                                                                                                   |


In [ ]:
# Define the dtypes explicitly to avoid issues with mismatched data types
dtypes = {
    'YEAR_MONTH': 'int64',  # Year and month should be treated as integers
    'REGIONAL_OFFICE_NAME': 'object',  # Treat as string (object) for name columns
    'REGIONAL_OFFICE_CODE': 'object',  # Treat as string (object) for code columns
    'ICB_NAME': 'object',  # Treat as string (object)
    'ICB_CODE': 'object',  # Treat as string (object)
    'PCO_NAME': 'object',  # Treat as string (object)
    'PCO_CODE': 'object',  # Treat as string (object)
    'PRACTICE_NAME': 'object',  # Treat as string (object)
    'PRACTICE_CODE': 'object',  # Treat as string (object)
    'ADDRESS_1': 'object',  # Treat as string (object)
    'ADDRESS_2': 'object',  # Treat as string (object)
    'ADDRESS_3': 'object',  # Treat as string (object)
    'ADDRESS_4': 'object',  # Treat as string (object)
    'POSTCODE': 'object',  # Treat as string (object) for postcode
    'BNF_CHEMICAL_SUBSTANCE': 'object',  # Ensure BNF_CHEMICAL_SUBSTANCE is treated as a string
    'CHEMICAL_SUBSTANCE_BNF_DESCR': 'object',  # Treat as string (object)
    'BNF_CODE': 'object',  # Ensure BNF_CODE is treated as a string
    'BNF_DESCRIPTION': 'object',  # Treat as string (object)
    'BNF_CHAPTER_PLUS_CODE': 'object',  # Treat as string (object)
    'QUANTITY': 'float64',  # Quantity should be numeric, float to handle decimals if any
    'ITEMS': 'int64',  # Items should be treated as integers
    'TOTAL_QUANTITY': 'float64',  # Total Quantity should be numeric, float
    'ADQUSAGE': 'float64',  # Ensure ADQUSAGE is treated as float
    'NIC': 'float64',  # NIC should be numeric, float
    'ACTUAL_COST': 'float64',  # ACTUAL_COST should be numeric, float
    'UNIDENTIFIED': 'object'  # Treat as string (object) for unidentified entries
}


In [ ]:
df_raw_Dec = dd.read_csv("data/epd_202412.csv", dtype=dtypes)

In [ ]:
df_raw_Nov = dd.read_csv("data/epd_202411.csv", dtype=dtypes)

In [ ]:
df_raw_Oct = dd.read_csv("data/epd_202410.csv", dtype=dtypes)

In [ ]:
df_raw_Sep = dd.read_csv("data/epd_202409.csv", dtype=dtypes)

In [ ]:
df_raw_Aug = dd.read_csv("data/epd_202408.csv", dtype=dtypes)

In [ ]:
df_raw_Jul = pd.read_csv("data/epd_202407.csv", dtype=dtypes)

In [ ]:
df_raw_Jun = dd.read_csv("data/epd_202406.csv", dtype=dtypes)

In [ ]:
df_raw_May = dd.read_csv("data/epd_202405.csv", dtype=dtypes)

In [ ]:
df_raw_Apr = dd.read_csv("data/epd_202404.csv", dtype=dtypes)

In [ ]:
df_raw_Mar = dd.read_csv("data/epd_202403.csv", dtype=dtypes)

In [ ]:
df_raw_Feb = dd.read_csv("data/epd_202402.csv", dtype=dtypes)

In [ ]:
df_raw_Jan = dd.read_csv("data/epd_202401.csv", dtype=dtypes)

In [ ]:
df_list = [df_raw_Jan, df_raw_Feb, df_raw_Mar, df_raw_Apr, df_raw_May, df_raw_Jun, df_raw_Jul, df_raw_Aug, df_raw_Sep, df_raw_Oct, df_raw_Nov, df_raw_Dec]

In [ ]:
del df_raw_Jan, df_raw_Feb, df_raw_Mar, df_raw_Apr, df_raw_May, df_raw_Jun, df_raw_Jul, df_raw_Aug, df_raw_Sep, df_raw_Oct, df_raw_Nov, df_raw_Dec

In [ ]:
gc.collect()

In [ ]:
df_raw = dd.concat(df_list, axis=0, ignore_index=True)

print("✅ Successfully merged all months!")

In [ ]:
del df_list

In [ ]:
gc.collect()

In [ ]:
df_raw.columns

#### Trim the data
Removing the columns that are either redundent or not useful for the purpose of the analysis.
Where redundent, I kept the CODE version rather than the NAME.
Reason: to decrease large amounts of data into the absolute necessary amounts

In [ ]:
df_raw = df_raw.drop(['REGIONAL_OFFICE_NAME', 'ICB_NAME', 'PCO_NAME', 'PRACTICE_NAME', 'ADDRESS_1', 'ADDRESS_2', 'ADDRESS_3', 'ADDRESS_4'], axis=1)

In [ ]:
# Convert Dask DataFrames to Pandas DataFrames and concatenate them
df_raw_computed = df_raw.compute()

print("✅ Dataset successfully converted to Pandas DataFrame!")

In [ ]:
# Save Dask DataFrame to Parquet with compression
# df_raw_computed.to_parquet('data/data_raw.parquet', compression='snappy')

print("✅ Data saved as Parquet with Snappy compression!")

In [3]:
# Load the Parquet file with Dask
data_loaded = dd.read_parquet('data/data_raw.parquet')

# Convert to Pandas DataFrame
data = data_loaded.compute()

print("✅ Data loaded successfully!")

✅ Data loaded successfully!
